# Mapping Connectivity Analysis

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import sample_colorscale
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Configuration

In [ ]:
# ============================================================================
# CONFIGURATION
# ============================================================================

#  Directories
PROJECT_ROOT = Path.cwd().resolve().parent
DATA_DIR = PROJECT_ROOT / "data"
OUTPUT_DIR = PROJECT_ROOT / "output"

# File Paths
DATA_FILE_PATH = OUTPUT_DIR / "transit_summary.csv"
SHAPEFILE_PATH = DATA_DIR / "suburbs/Locality_Boundaries.shp"

# Figure Path
FIGURE_PATH = "assets/figures"

# Route type mapping
ROUTE_TYPE_MAP = {
    0: 'Tram',
    2: 'Rail', 
    3: 'Bus',
    4: 'Ferry'
}

# Color palette for modes
MODE_COLORS = {
    'Tram': '#7D9B76', 
    'Bus': '#4E79A7',
    'Rail': '#59A14F',
    'Ferry': '#F28E2B'
}

DAY_COLORS = {
    'Weekday': "#3F3F74",
    'Weekend': "#B55D60"
}

# Analysis period text
DEFAULT_ANALYSIS_PERIOD = "(Typical week service patterns January 12th (Monday) to January 18th (Sunday), 2026)"

# Figure styling function
def apply_plotly_style(fig, title=None, x_title=None, y_title=None, analysis_period_text=''):
    """Apply consistent formatting to a Plotly figure."""
    
    fig.update_layout(
        template='plotly_white',
        font=dict(size=12, color='black'),
        title={'text': f"{title}<br><span style='font-size:12px; color:gray;'>{analysis_period_text}</span>", 
               'x':0.5, 
               'xanchor':'center', 
               'font': {'size':14, 
                        'color':'black'}},
        hoverlabel=dict(
            bgcolor="white",
            font_size=13,
            font_family="Arial"
        ),
        margin={"r": 0, "l": 0, "b": 0}
    )

    fig.update_xaxes(
        title_text=x_title,
        showline=True,
        linecolor='black',
        linewidth=1.5,
        mirror=True
    )

    fig.update_yaxes(
        title_text=y_title,
        showline=True,
        linecolor='black',
        linewidth=1.5,
        mirror=True,
        fixedrange=False
    )

    return fig

In [ ]:
# ============================================================================
# Filters
# ============================================================================
locality_of_interest = 'Brisbane City'

## Data Load

In [ ]:
raw_df = pd.read_csv(DATA_FILE_PATH)
raw_df['from_loc_code'] = raw_df['from_loc_code'].astype(str)
raw_df['to_loc_code'] = raw_df['to_loc_code'].astype(str)

In [ ]:
localities = gpd.read_file(SHAPEFILE_PATH)

## Data Preparation

In [ ]:
# Apply Filters
data = raw_df[
        (raw_df['from_locality'] == locality_of_interest) |
        (raw_df['to_locality'] == locality_of_interest)
    ].copy()

In [ ]:
# Ensure correct dtypes
data['weekday'] = data['weekday'].astype(int)
data['weekend'] = data['weekend'].astype(int)

# Mode labels
data['mode'] = data['route_type'].map(ROUTE_TYPE_MAP)
data = data.drop(columns=['route_type'])

# Day
data['Day'] = np.where(data['weekday'] == 1, 'Weekday', 'Weekend')

# Determine the other locality involved in the trip
data['locality'] = np.where(
    data['from_locality'] == locality_of_interest,
    data['to_locality'],
    data['from_locality']
)

# Determine code of the other locality involved in the trip
data['locality_code'] = np.where(
    data['from_locality'] == locality_of_interest,
    data['to_loc_code'],
    data['from_loc_code']
)

## Analysis

## Localities with Direct Connections

In [ ]:
def map_localities_with_direct_connections(df, localities_gdf, locality_of_interest, ANALYSIS_PERIOD, FIGURE_PATH):
    """
    Plot localities directly connected to the locality of interest by any mode of transport.
    """
    config = {'scrollZoom': True}

    df = df[df['to_locality']==locality_of_interest]
    df = df[df['Day'] == 'Weekday']
    df = df.groupby(['locality_code', 'locality']).agg({'number_of_routes': 'sum'}).reset_index()

    map_gdf = gpd.GeoDataFrame(
        df.merge(
            localities_gdf[['loc_code', 'geometry']],
            left_on='locality_code',
            right_on='loc_code',
            how='left'
        ).drop(columns=['loc_code']),
        crs=localities.crs
    )
    
    # Create the map figure
    fig = px.choropleth_mapbox(
        map_gdf,
        geojson=map_gdf.__geo_interface__,
        locations=map_gdf.index,
        color='number_of_routes',
        color_continuous_scale='RdYlGn',
        center={"lat": -26.95, "lon": 153.1},
        zoom=8,
        height=800,
        mapbox_style='carto-positron',
        opacity=0.7,
        labels={'number_of_routes': 'Number of routes'}
    )

    fig.update_traces(
        hovertemplate=(
            "Locality: %{customdata[0]}<br>" +
            "Number of routes: %{z}<extra></extra>"
        ),
        customdata=map_gdf[['locality']].values  # pass locality as customdata
    )


    fig = apply_plotly_style(fig, title=f"Localities with direct connections to {locality_of_interest} (Weekday) <br><span style='font-size:12px; color:gray;'>{ANALYSIS_PERIOD}</span>")


    fig.show(config=config)
    fig.write_html(f"{FIGURE_PATH}/localities_with_direct_connections.html", config=config)

map_localities_with_direct_connections(df=data, localities_gdf=localities, locality_of_interest=locality_of_interest, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD, FIGURE_PATH=FIGURE_PATH)


In [ ]:
def direct_connections_by_mode(df, locality_of_interest, MODE_COLORS, DAY_COLORS, ANALYSIS_PERIOD, FIGURE_PATH):
    """
    Analyze and plot the number of distinct localities directly connected to the locality of interest by mode of transport.
    """
    df = df[df['to_locality']==locality_of_interest]
    # Distinct localities with direct connections to the locality of interest 
    print('Number of distinct localities:', len(df['locality'].unique()))

    # Localities connected by each mode
    by_mode = df.groupby('mode')['locality'].nunique().reset_index()
    # Localities connected by day
    by_day = df.groupby('Day')['locality'].nunique().reset_index()
    # Localities connected by mode and day
    by_mode_day = df.groupby(['mode', 'Day'])['locality'].nunique().reset_index()

    # Group by mode and count distinct localities
    by_mode_day = df.groupby(['mode', 'Day'])['locality'].nunique().reset_index()
    by_mode_day = by_mode_day.sort_values('locality', ascending=False)

    # Plot
    fig = px.bar(
        by_mode.sort_values('locality', ascending=False),
        x='mode',
        y='locality',
        color='mode',
        color_discrete_map=MODE_COLORS,
        text='locality',
        labels={'mode': 'Mode'},
    )

    fig.update_traces(
        hovertemplate=(
            "Mode: %{x}<br>"
            "Number of Localities: %{y}<br>"
        )
    )

    fig = apply_plotly_style(
        fig,
        title=f'Localities with Direct Connections with {locality_of_interest} by Mode',
        x_title='Mode of Transport',
        y_title='Number of Localities',
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig.show()
    fig.write_html(f"{FIGURE_PATH}/direct_connections_by_mode.html")


    fig1 = px.bar(
        by_mode_day,
        x='mode',
        y='locality',
        color='Day',
        color_discrete_map=DAY_COLORS,
        barmode='group',
        text='locality',
        labels={'mode': 'Mode'},
    )

    fig1.update_traces(
        hovertemplate=(
            "Mode: %{x}<br>"
            "Number of Localities: %{y}<br>"
        )
    )

    fig1 = apply_plotly_style(
        fig1,
        title=f'Localities with Direct Connections with {locality_of_interest} by Mode and Day',
        x_title='Mode of Transport',
        y_title='Number of Localities',
        analysis_period_text=ANALYSIS_PERIOD
    )
    fig1.show()
    fig1.write_html(f"{FIGURE_PATH}/direct_connections_by_mode_and_day.html")

    # By mode combo
    locality_mode_combo = (
        df.groupby('locality')['mode']
        .apply(
            lambda x: (
                f"{list(set(x))[0]} Only"
                if len(set(x)) == 1
                else ' + '.join(sorted(set(x)))
            )
        )
        .reset_index(name='mode_combo')
    )

    by_mode_combo = (
        df.merge(locality_mode_combo, on='locality')
        .groupby(['mode_combo', 'Day'])['locality']
        .nunique()
        .reset_index()
        .pivot(index='mode_combo', columns='Day', values='locality')
        .reset_index()
    )

    by_mode_combo['pct_diff'] = (
        (by_mode_combo['Weekday'] - by_mode_combo['Weekend'])
        / by_mode_combo['Weekday'] * 100
    ).round(1)

    fig2 = px.bar(
        by_mode_combo.sort_values('pct_diff', ascending=False),
        x='mode_combo',
        y='pct_diff',
        text='pct_diff',
        labels={'mode_combo': 'Mode Combo', 'pct_diff': 'Percentage Change <br >in Localities'},
    )

    fig2.update_traces(
        texttemplate='%{text:.1f}%',
        hovertemplate=(
            "Mode Combo: %{x}<br>"
            "Percentage Change in Localities: %{y:,.1f}%<br>"
        )
    )

    fig2 = apply_plotly_style(
        fig2,
        title=f'Percentage Change in Localities with Direct Connections to {locality_of_interest} on Weekend by Mode',
        x_title='Mode Combo',
        y_title='Percentage Change in Localities',
        analysis_period_text=ANALYSIS_PERIOD
    )
    fig2.show()
    fig2.write_html(f"{FIGURE_PATH}/pct_change_localities_by_mode_combo.html")

    # Statistics
    pct_change = by_mode_day.pivot(index='mode', columns='Day', values='locality')
    pct_change['pct_drop'] = (pct_change['Weekday'] - pct_change['Weekend']) / pct_change['Weekday'] * 100
    for mode in pct_change.index:
        print(f"Percentage change in localities for {mode}: {pct_change.loc[mode, 'pct_drop']:.2f}%")

direct_connections_by_mode(df=data, locality_of_interest=locality_of_interest, MODE_COLORS=MODE_COLORS, DAY_COLORS=DAY_COLORS, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD, FIGURE_PATH=FIGURE_PATH)

In [ ]:
def direct_connections_by_mode_combo(df, locality_of_interest, ANALYSIS_PERIOD, FIGURE_PATH):
    """
    Analyze and plot the number of distinct localities directly connected to the locality of interest by mode of transport.
    """
    df = df[df['to_locality']==locality_of_interest]
    locality_mode_combo = (
        df.groupby('locality')['mode']
        .apply(
            lambda x: (
                f"{list(set(x))[0]} Only"
                if len(set(x)) == 1
                else ' + '.join(sorted(set(x)))
            )
        )
        .reset_index(name='mode_combo')
    )

    combo_counts = (
        locality_mode_combo
        .groupby('mode_combo')
        .size()
        .reset_index(name='locality_count')
        .sort_values('locality_count', ascending=False)
    )
    # Plot
    fig = px.pie(
        combo_counts,
        names='mode_combo',
        values='locality_count',
        labels={'mode_combo': 'Mode', 'locality_count': 'Number of Localities'},
    )

    fig.update_traces(
        hovertemplate=(
            "Mode: %{label}<br>"
            "Number of Localities: %{value}<br>"
        )
    )

    fig = apply_plotly_style(
        fig,
        title=f'Localities with Direct Connections with {locality_of_interest} by Mode',
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig.show()
    fig.write_html(f"{FIGURE_PATH}/direct_connections_by_mode_combo.html")

    return locality_mode_combo

locality_mode_combo = direct_connections_by_mode_combo(df=data, locality_of_interest=locality_of_interest, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD, FIGURE_PATH=FIGURE_PATH)

# Travel Time Analysis

In [ ]:
def travel_time_distribution(df, locality_of_interest, MODE_COLORS, FIGURE_PATH):
    """
    Analyze and plot the distribution of travel times for direct connections to the locality of interest by mode of transport.
    """
    df = df[df['Day']=='Weekday']
    df = df[df['to_locality']==locality_of_interest]
    ANALYSIS_PERIOD = "(Typical weekday service patterns January 12th (Monday) to January 18th (Sunday), 2026)"
    # Plot
    fig = px.ecdf(
        df,
        x='travel_time_median',
        color='mode',
        color_discrete_map=MODE_COLORS,
        marginal="histogram",
        labels={'travel_time_median': 'Travel Time (min)', 'mode': 'Mode'},
    )

    for trace in fig.data:
        if trace.type == "scatter":   # ECDF traces
            trace.hovertemplate = (
                "Mode: %{fullData.name}<br>"
                "Travel Time (min): %{x}<br>"
                "Proportion of Localities: %{y:.1%}<extra></extra>"
            )
        else:                         # Histogram traces
            trace.hovertemplate = (
                "Mode: %{fullData.name}<br>"
                "Travel Time (min): %{x}<br>"
                "Number of Localities: %{y}<extra></extra>"
            )

    fig = apply_plotly_style(
        fig,
        title=f'Distribution of Travel Times for Direct Connections to {locality_of_interest} (Weekday)',
        x_title='Travel Time (minutes)',
        y_title='Proportion Of Localities',
        analysis_period_text=ANALYSIS_PERIOD
    )

    for axis_name in fig.layout:
        if axis_name.startswith("xaxis"):
            axis = fig.layout[axis_name]
            if axis.anchor == "y2":
                axis.title.text = ""  # remove marginal x-axis label
        if axis_name.startswith("yaxis"):
            axis = fig.layout[axis_name]
            if hasattr(axis, "domain") and axis.domain[1] > 0.8:
                axis.title.text = ""         
            else:
                axis.title.text = "Proportion of Localities"

    fig.show()
    fig.write_html(f"{FIGURE_PATH}/travel_time_distribution.html")

travel_time_distribution(df=data, locality_of_interest=locality_of_interest, MODE_COLORS=MODE_COLORS, FIGURE_PATH=FIGURE_PATH)

## Shared Mode Comparison

In [ ]:
def shared_mode_comparison(df, ANALYSIS_PERIOD, FIGURE_PATH):
    df = df[df['to_locality']==locality_of_interest]
    for day in ['Weekday']:
        df_br = df[df['Day'] == day]    
        df_br = df_br[df_br['mode'].isin(['Bus', 'Rail'])]
        
        localities_both = (
            df_br.groupby(['locality'])['mode']
            .nunique()
            .loc[lambda x: x == 2]
            .index
        )

        df_br = df_br[df_br['locality'].isin(localities_both)]
        pivot = (
            df_br
            .groupby(['locality', 'Day', 'mode', 'locality_code'])['travel_time_median']
            .median()
            .reset_index()
            .pivot(
                index=['locality', 'locality_code', 'Day'],
                columns='mode',
                values='travel_time_median'
            )
            .dropna()
            .reset_index()
        )
        
        pivot['bus_minus_rail'] = pivot['Bus'] - pivot['Rail']

        df_cum = pivot[['Day', 'bus_minus_rail', 'locality']].copy()

        # Sort by Bus − Rail difference
        df_cum = df_cum.sort_values(['Day', 'bus_minus_rail'])
        # Cumulative count per Day
        df_cum['cum_count'] = df_cum.groupby('Day').cumcount() + 1
        # Cumulative percentage per Day
        df_cum['cum_percent'] = df_cum['cum_count'] / df_cum.groupby('Day')['cum_count'].transform('max') * 100

        fig = make_subplots(specs=[[{"secondary_y": True}]])

        for day in df_cum['Day'].unique():
            df_day = df_cum[df_cum['Day'] == day]
            
            fig.add_trace(
                go.Scatter(
                    x=df_day['bus_minus_rail'],
                    y=df_day['cum_percent'],
                    mode='lines+markers',
                    name=f'Cumulative % ({day})',
                    line=dict(width=2),
                    hovertemplate=(
                        'Time difference: %{x} min<br>'
                        'Percentage of localities: %{y:.1f}%<br>'
                        'Number of localities: %{customdata}'
                        '<extra></extra>'
                    ),
                    customdata=df_day['cum_count']
                ),
                secondary_y=False 
            )

        fig.update_layout(
            title='Bus − Rail Travel Time Difference: Cumulative % of Localities',
            xaxis_title='Bus − Rail Travel Time Difference (min)',
            yaxis_title='Cumulative % of Localities',
            hovermode='x unified'
        )

        # Configure right y-axis
        fig.update_yaxes(
            title_text='Number of Localities',
            secondary_y=True
        )

        # Add zero reference line
        fig.add_vline(
            x=0,
            line_dash='dash',
            line_color='black',
            annotation_text='Bus = Rail',
            annotation_position='top left'
        )

        fig.update_layout(
            title='Bus − Rail Travel Time Difference: Cumulative % and Count of Localities',
            xaxis_title='Bus − Rail Travel Time Difference (min)',
            legend_title='Metric'
        )

        fig.update_yaxes(
            title_text='Cumulative % of Localities',
            secondary_y=False,
            range=[0, 100]
        )
        fig.update_yaxes(
            title_text='Number of Localities',
            secondary_y=True
        )

        fig = apply_plotly_style(
            fig,
            title=f'Bus–Rail Travel Time Difference for Localities Connected to {locality_of_interest} by Both Modes ({day})',
            x_title='Bus Travel Time Minus Rail Travel Time (minutes)',
             y_title='Percentage of Localities',
            analysis_period_text=ANALYSIS_PERIOD
        )
        fig.update_layout(hovermode='x')


        fig.show()
        fig.write_html(f"{FIGURE_PATH}/shared_mode_comparison_{day}.html")

        #Dumbell figure
        pivot = pivot.sort_values('bus_minus_rail', ascending=True)
        
        # Min / max for normalization
        vmin = pivot['bus_minus_rail'].min()
        vmax = pivot['bus_minus_rail'].max()

        def diff_to_color(value, colorscale='RdYlGn_r'):
            """
            Maps bus_minus_rail value to a color
            Green (low) -> Red (high)
            """
            t = (value - vmin) / (vmax - vmin) if vmax > vmin else 0.5
            return sample_colorscale(colorscale, t)[0]

        fig2 = go.Figure()

        for _, row in pivot.iterrows():
            fig2.add_trace(
                go.Scatter(
                    x=[row['Rail'], row['Bus']],
                    y=[row['locality'], row['locality']],
                    mode='lines',
                    line=dict(
                    color=diff_to_color(row['bus_minus_rail']),
                    width=4
                ),
                hovertemplate=
                    'Locality: %{y}<br>'
                    'Bus − Rail: %{customdata:.1f} min<extra></extra>',
                customdata=[row['bus_minus_rail']],
                showlegend=False
            )
        )

        # Rail points
        fig2.add_trace(
            go.Scatter(
                x=pivot['Rail'],
                y=pivot['locality'],
                mode='markers',
                name='Rail',
                marker=dict(size=6),
                hovertemplate=
                    'Locality: %{y}<br>'
                    'Mode: Rail<br>'
                    'Travel Time: %{x} min<extra></extra>'
            )
        )

        # Bus points
        fig2.add_trace(
            go.Scatter(
                x=pivot['Bus'],
                y=pivot['locality'],
                mode='markers',
                name='Bus',
                marker=dict(size=6),
                hovertemplate=
                    'Locality: %{y}<br>'
                    'Mode: Bus<br>'
                    'Median time: %{x} min<extra></extra>'
            )
        )

        # Add colorbar
        fig2.add_trace(
            go.Scatter(
                x=[None],
                y=[None],
                mode='markers',
                marker=dict(
                    colorscale='RdYlGn_r',
                    cmin=vmin,
                    cmax=vmax,
                    color=[vmin, vmax],
                    showscale=True,
                    colorbar=dict(
                        title='Time Difference between <br> Bus and Rail (minutes)',
                        orientation='v',
                        thickness=8,
                        len=0.5,
                        y=0.5,               
                                 
                    )
                ),
                hoverinfo='skip',
                showlegend=False
            )
        )

        fig2.update_layout(
            xaxis_title='Median Travel Time (min)',
            yaxis_title='Locality',
            hovermode='closest',
            legend_title='Mode',
            height=max(400, pivot['locality'].nunique() * 20)
        )
        


        fig2 = apply_plotly_style(
            fig2,
            title=f'Bus–Rail Travel Time Difference for Localities Connected to {locality_of_interest} by Both Modes ({day})',
            x_title='Travel Time (minutes)',
            analysis_period_text=ANALYSIS_PERIOD
        )


        fig2.show()
        fig2.write_html(f"{FIGURE_PATH}/shared_mode_dumbell_{day}.html")

        return pivot

mode_difference = shared_mode_comparison(df=data, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD, FIGURE_PATH=FIGURE_PATH)

In [ ]:
def mode_difference_map(mode_difference, localities, locality_of_interest, ANALYSIS_PERIOD, FIGURE_PATH):
    """
    Create a map visualizing the bus minus rail travel time difference for localities connected by both modes.
    """    
    def mode_faster_text(row):
        if row['bus_minus_rail'] < 0:
            return f"Bus is faster by {abs(row['bus_minus_rail']):.0f} min"
        elif row['bus_minus_rail'] > 0:
            return f"Train is faster by {abs(row['bus_minus_rail']):.0f} min"
        else:
            return "Bus and Train have same time"
    mode_difference['faster_text'] = mode_difference.apply(mode_faster_text, axis=1)
    # Map
    map_gdf = gpd.GeoDataFrame(
        mode_difference.merge(
            localities[['loc_code', 'geometry']],
            left_on='locality_code',
            right_on='loc_code',
            how='left'
        ).drop(columns=['loc_code']),
        crs=localities.crs
    )
    map_gdf = map_gdf.to_crs(epsg=4326)
    map_gdf = map_gdf.reset_index(drop=True)
    geojson = map_gdf.__geo_interface__

    interest_geom = (
            localities
            .loc[localities['locality'] == locality_of_interest, 'geometry']
            .to_crs(map_gdf.crs)
            .iloc[0]
        )
    interest_gdf = gpd.GeoDataFrame(
            {'locality': [locality_of_interest], 'geometry': [interest_geom]},
            crs=map_gdf.crs
        )

    # Plot Localities
    fig = px.choropleth_mapbox(
        map_gdf,
        geojson=geojson,
        locations=map_gdf.index,
        color='bus_minus_rail',
        color_continuous_scale='RdYlGn_r',
        mapbox_style='carto-positron',
        zoom=7,
        center={
            'lat': map_gdf.geometry.centroid.y.mean(),
            'lon': map_gdf.geometry.centroid.x.mean()
        },
        opacity=0.7,
    )


    fig.update_traces(
        hovertemplate=
            "<b>%{customdata[0]}</b><br>" +
            "Bus travel time: %{customdata[1]:.0f} min<br>" +
            "Rail travel time: %{customdata[2]:.0f} min<br>" +
            "<b>%{customdata[3]}</b>" +  # dynamic faster text
            "<extra></extra>",
        customdata=map_gdf[['locality', 'Bus', 'Rail', 'faster_text']].values
    )

    fig.add_trace(
        go.Choroplethmapbox(
            geojson=interest_gdf.__geo_interface__,
            locations=interest_gdf.index,
            z=[1],  # dummy, just to enable color
            showscale=False,  
            marker_opacity=0.7,
            marker_line_width=1.5,
            marker_line_color='black',
            colorscale=[[0, 'gray'], [1, 'gray']], 
            name=locality_of_interest,
            hovertemplate=f"<b>{locality_of_interest}</b><br>Locality of interest<extra></extra>"
        )
    )

    # Move colorbar to the bottom
    fig.update_layout(
        coloraxis_colorbar=dict(
            title=None,
            orientation='h',   
            thickness=10,     
            len=0.6,           
            x=0.5,             
            xanchor='center',
            y=0,               
            yanchor='bottom',
        ),
        mapbox=dict(
            style='carto-positron',
            center={"lat": map_gdf.geometry.centroid.y.mean(),
                    "lon": map_gdf.geometry.centroid.x.mean()},
            zoom=10,
        ),
        dragmode='pan',  # enables zooming/panning
    )

    # Draw horizontal colorbar as shape
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                x0=0.2, x1=0.8,  
                y0=-0.05, y1=-0.045, 
                xref="paper",
                yref="paper",
                line=dict(width=0),
            )
        ]
    )

    # Add left/right labels
    color_scale = list(reversed(list(px.colors.diverging.RdYlGn)))
    fig.add_annotation(
        x=0.15, y=0, xref="paper", yref="paper",
        text="Bus is faster", showarrow=False, font=dict(size=12, color = color_scale[0])
    )
    fig.add_annotation(
        x=0.86, y=0, xref="paper", yref="paper",
        text="Train is faster", showarrow=False, font=dict(size=12, color = color_scale[-1])
    )


    fig = apply_plotly_style(
        fig,
        title=f'Bus–Rail Travel Time Difference for Localities Connected to {locality_of_interest} by Both Modes (Weekday)',
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig.show(config = {'scrollZoom': True})
    fig.write_html(f"{FIGURE_PATH}/shared_mode_map.html", config = {'scrollZoom': True})

mode_difference_map(mode_difference, localities, locality_of_interest, DEFAULT_ANALYSIS_PERIOD, FIGURE_PATH)


## Locality within 30 minutes

In [ ]:
def localities_within_minutes_slider(df, locality_of_interest, MODE_COLORS, ANALYSIS_PERIOD):

    df = df[(df['Day'] == 'Weekday') & (df['to_locality'] == locality_of_interest)]

    # Base counts
    by_mode = df.groupby('mode')['locality'].nunique().reset_index()
    by_mode = by_mode.sort_values('locality', ascending=False)

    # Precompute for all minutes (0–60 or whatever range you want)
    minute_range = range(0, 61, 5)
    minute_list = list(minute_range)
    default_minute = 30
    default_index = minute_list.index(default_minute)

    print(f"Number of localities within {default_minute} minutes: {df[df['travel_time_median']<=default_minute]['locality_code'].nunique()} ({df[df['travel_time_median']<=default_minute]['locality_code'].nunique()/df['locality_code'].nunique():.1%})")

    frames = []
    for m in minute_range:
        within = df[df['travel_time_median'] <= m]
        within_by_mode = within.groupby('mode')['locality'].nunique().reindex(by_mode['mode']).fillna(0)

        percentage = (within_by_mode.values / by_mode['locality'].values) * 100

        frames.append({
            "minutes": m,
            "within": within_by_mode.values,
            "percentage": percentage
        })

    # Figure
    fig = go.Figure()

    # Initial frame 
    initial = frames[default_index]

    fig.add_trace(go.Bar(
        x=by_mode['mode'],
        y=initial['within'],   # correct bar height
        marker_color=[MODE_COLORS[m] for m in by_mode['mode']],
        customdata=np.stack([
            by_mode['mode'],
            initial['within'],       # <-- MUST match slider
            initial['percentage']    # <-- MUST match slider
        ], axis=-1),
        text=initial['within'],      # <-- MUST match slider
        texttemplate='%{customdata[1]} (%{customdata[2]:.1f}%)',
        hovertemplate="<b>%{customdata[0]}</b><br>" +
                    "Number of localities: %{customdata[1]}<br>" +
                    "Percentage: %{customdata[2]:.1f}%<extra></extra>"
    ))

    # Slider
    steps = []
    for i, f in enumerate(frames):
        step = dict(
            method="update",
            args=[
                {
                    "y": [f['within']],
                    "customdata": [np.stack([by_mode['mode'], f['within'], f['percentage']], axis=-1)],
                    "text": [f['within']]
                },
                {
                    "title": {
                        "text": f"Localities Reachable Within {f['minutes']} Minutes from {locality_of_interest}<br><span style='font-size:12px; color:gray;'>{ANALYSIS_PERIOD}</span>",
                        'x':0.5, 
                        'xanchor':'center', 
                        'font': {'size':14, 
                                    'color':'black'},
                    }
                }
            ],
            label=str(f['minutes'])
        )
        steps.append(step)

    sliders = [dict(
        active=default_index,
        currentvalue={"prefix": "Minutes: "},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(
        sliders=sliders,
    )

    fig = apply_plotly_style(
        fig,
        title=f"{locality_of_interest} Reachable Within {default_minute} Minutes(Weekday)",
        y_title="Number of Localities",
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig.update_yaxes(range=[0, by_mode['locality'].max()])

    fig.show()
    fig.write_html(f"{FIGURE_PATH}/localities_within_minutes.html")

localities_within_minutes_slider(df=data, locality_of_interest=locality_of_interest, MODE_COLORS=MODE_COLORS, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD)

In [ ]:
def localities_within_30min_map_two_layers(
    df,
    localities_gdf,
    locality_of_interest,
    ANALYSIS_PERIOD,
    FIGURE_PATH,
    minutes=30,
    modes_to_plot=["Bus", "Rail"],
):
    """
    Two-layer map:
    - Layer 1: reachable polygons coloured by travel_time_median
    - Layer 2: unreachable polygons in a single grey colour
    """

    config = {'scrollZoom': True}
    df = df[(df["Day"] == "Weekday") & (df["to_locality"] == locality_of_interest)]

    for mode in modes_to_plot:
        map_df = df[df["mode"] == mode]
        # Split reachable vs unreachable
        reachable = map_df[(map_df["travel_time_median"] <= minutes) & (map_df["mode"] == mode)]
        unreachable = map_df[(map_df["travel_time_median"] > minutes) & (map_df["mode"] == mode)]

        # Merge polygons
        reachable_gdf = reachable.merge(
            localities_gdf[["loc_code", "geometry"]],
            left_on="locality_code",
            right_on="loc_code",
            how="left"
        )

        unreachable_gdf = unreachable.merge(
            localities_gdf[["loc_code", "geometry"]],
            left_on="locality_code",
            right_on="loc_code",
            how="left"
        )

        # Convert to GeoDataFrames
        reachable_gdf = gpd.GeoDataFrame(reachable_gdf, crs=localities_gdf.crs)
        unreachable_gdf = gpd.GeoDataFrame(unreachable_gdf, crs=localities_gdf.crs)

        # Compute centroids for map centering
        reachable_gdf["centroid_lat"] = reachable_gdf.geometry.centroid.y
        reachable_gdf["centroid_lon"] = reachable_gdf.geometry.centroid.x
        centre_lat = reachable_gdf["centroid_lat"].median()
        centre_lon = reachable_gdf["centroid_lon"].median()

        # LAYER 1: REACHABLE (colour scale)
        fig = px.choropleth_mapbox(
            reachable_gdf,
            geojson=reachable_gdf.__geo_interface__,
            locations=reachable_gdf.index,
            color="travel_time_median",
            color_continuous_scale='RdYlGn_r',
            center={"lat": centre_lat, "lon": centre_lon},
            zoom=8,
            mapbox_style="carto-positron",
            opacity=0.85,
            # height=800,
            labels={"travel_time_median": "Median travel time (min)"}
        )

        fig.update_traces(
            hovertemplate=(
                "Locality: %{customdata[0]}<br>"
                "Travel Time: %{z} min<extra></extra>"
            ),
            customdata=reachable_gdf[["locality"]].values
        )

        # LAYER 2: UNREACHABLE
        unreachable_color = "#6d81b4"
        fig.add_choroplethmapbox(
            geojson=unreachable_gdf.__geo_interface__,
            locations=unreachable_gdf.index,
            z=[1] * len(unreachable_gdf),
            colorscale=[[0, unreachable_color], [1, unreachable_color]],
            showscale=False,
            marker_opacity=0.5,
            marker_line_width=0.5,
            hovertemplate="Locality: %{customdata[0]}<extra></extra>",
            customdata=unreachable_gdf[["locality"]].values
        )
       
        # LAYER3: LOCALITY OF INTEREST
        interest_geom = (
            localities
            .loc[localities['locality'] == locality_of_interest, 'geometry']
            .to_crs(localities.crs)
            .iloc[0]
        )
        interest_gdf = gpd.GeoDataFrame(
                {'locality': [locality_of_interest], 'geometry': [interest_geom]},
                crs=localities.crs
            )
        
        fig.add_trace(
            go.Choroplethmapbox(
                geojson=interest_gdf.__geo_interface__,
                locations=interest_gdf.index,
                z=[1],  # dummy, just to enable color
                showscale=False,  
                marker_opacity=0.7,
                marker_line_width=1.5,
                marker_line_color='black',
                colorscale=[[0, 'gray'], [1, 'gray']], 
                name=locality_of_interest,
                hovertemplate=f"<b>{locality_of_interest}</b><br>Locality of interest<extra></extra>"
            )
        )

        # Move colorbar to the bottom
        fig.update_layout(
            coloraxis_colorbar=dict(
                title='Travel Time',
                orientation='h',   
                thickness=10,     
                len=0.6,           
                x=0.5,             
                xanchor='center',
                y=0,               
                yanchor='bottom',
            )
        )
            
        # TITLE
        fig.update_layout(
            title={
                "text": (
                    f"{locality_of_interest} Reachable within {minutes} minutes by {mode} (Weekday)"
                    f"<br><span style='font-size:12px; color:gray;'>{ANALYSIS_PERIOD}</span>"
                ),
                "x": 0.5,
                "xanchor": "center",
                "font": {"size": 14, "color": "black"}
            },
            margin={"r": 0, "t": 40, "l": 0, "b": 0}
        )

        fig.show(config=config)
        fig.write_html(f"{FIGURE_PATH}/localities_within_{minutes}_minutes_{mode}_map.html", config=config)

localities_within_30min_map_two_layers(df=data, localities_gdf=localities, locality_of_interest=locality_of_interest, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD, FIGURE_PATH=FIGURE_PATH, minutes=30, modes_to_plot=["Bus", "Rail"])

# Number Trips Analysis

### Weekday vs Weekend Service Drop

In [ ]:
def plot_weekday_weekend_service_drop(df, localities, locality_of_interest, ANALYSIS_PERIOD):
    """
    Computes and plots the Weekday vs Weekend service drop
    for all origins serving locality of interest
    """
    config = {'scrollZoom': True}
    # Filter to locality of interest
    df = df[df["to_locality"] == locality_of_interest]

    #Compute weekday vs weekend trips
    summary = (
        df.groupby(["locality", "locality_code", "mode", "Day"])["number_of_trips_per_day"]
        .sum()
        .reset_index()
        .pivot(index=["locality", "locality_code", "mode"], columns="Day", values="number_of_trips_per_day")
        .fillna(0)
        .rename(columns={"Weekday": "mode_weekday_trips", "Weekend": "mode_weekend_trips"})
    )
    summary['weekday_trips'] = summary.groupby('locality').transform('sum')['mode_weekday_trips']
    summary['weekend_trips'] = summary.groupby('locality').transform('sum')['mode_weekend_trips']

    summary["drop"] = summary["weekday_trips"] - summary["weekend_trips"]
    summary["pct_drop"] = (
        summary["drop"] / summary["weekday_trips"].replace(0, np.nan)* 100
    ).round(1).clip(lower=0)

    summary["mode_drop"] = summary["mode_weekday_trips"] - summary["mode_weekend_trips"]
    summary["mode_pct_drop"] = (
        (summary["mode_drop"] /
        summary["mode_weekday_trips"].replace(0, np.nan))
        * 100
    ).round(1).clip(lower=0)

    summary = summary.reset_index()
    
    summary = summary.merge(
            localities[["loc_code", "geometry"]],
            left_on="locality_code",
            right_on="loc_code",
            how="left"
        )
    summary = gpd.GeoDataFrame(summary, crs=localities.crs)
    centre_lat = localities[localities["locality"] == locality_of_interest].geometry.centroid.y.values[0]
    centre_lon = localities[localities["locality"] == locality_of_interest].geometry.centroid.x.values[0]

    summary_all = summary.copy()
    summary_bus = summary[summary["mode"] == "Bus"]
    summary_train = summary[summary["mode"] == "Rail"]
    summary_ferry = summary[summary["mode"] == "Ferry"]

    fig = px.choropleth_mapbox(
        summary_all,
        geojson=summary_all.__geo_interface__,
        locations=summary_all.index,
        color="pct_drop",
        color_continuous_scale="RdYlGn_r",
        range_color=[0, 100],
        center={"lat": centre_lat, "lon": centre_lon},
        zoom=9,
        mapbox_style="carto-positron",
        opacity=0.7,
        labels={"pct_drop": "Percent drop in <br> number of trips per day"}
    )

    fig.update_traces(
        hovertemplate="Locality: %{customdata[0]}<br>Trips per Day Reduced by: %{z}%<extra></extra>",
        customdata=summary_all[["locality"]].values,
        visible=True 
    )

    # BUS
    fig.add_choroplethmapbox(
        geojson=summary_bus.__geo_interface__,
        locations=summary_bus.index,
        z=summary_bus["mode_pct_drop"],
        colorscale="RdYlGn_r",
        marker_opacity=0.7,
        zmin=0,
        zmax=100,
        marker_line_width=1,
        customdata=summary_bus[["locality"]].values,
        hovertemplate="Locality: %{customdata[0]}<br>Trips per Day Reduced by: %{z}%<extra></extra>",
        colorbar=dict(title="Percent drop in <br> number of trips per day"),
        visible=False
    )
    

    # TRAIN
    fig.add_choroplethmapbox(
        geojson=summary_train.__geo_interface__,
        locations=summary_train.index,
        z=summary_train["mode_pct_drop"],
        colorscale="RdYlGn_r",
        marker_opacity=0.7,
        zmin=0,
        zmax=100,
        marker_line_width=1,
        customdata=summary_train[["locality"]].values,
        hovertemplate="Locality: %{customdata[0]}<br>Trips per Day Reduced by: %{z}%<extra></extra>",
        colorbar=dict(title="Percent drop in <br> number of trips per day"),
        visible=False
    )

    # FERRY
    fig.add_choroplethmapbox(
        geojson=summary_ferry.__geo_interface__,
        locations=summary_ferry.index,
        z=summary_ferry["mode_pct_drop"],
        colorscale="RdYlGn_r",
        marker_opacity=0.7,
        zmin=0,
        zmax=100,
        marker_line_width=1,
        customdata=summary_ferry[["locality"]].values,
        hovertemplate="Locality: %{customdata[0]}<br>Trips per Day Reduced by: %{z}%<extra></extra>",
        colorbar=dict(title="Percent drop in <br> number of trips per day"),
        visible=False
    )

    fig.update_layout(
        updatemenus=[
            dict(
                buttons=[
                    dict(label="All Modes", method="update",
                        args=[{"visible": [True, False, False, False]}]),
                    dict(label="Bus", method="update",
                        args=[{"visible": [False, True, False, False]}]),
                    dict(label="Train", method="update",
                        args=[{"visible": [False, False, True, False]}]),
                    dict(label="Ferry", method="update",
                        args=[{"visible": [False, False, False, True]}]),
                ],
                direction="down",
                showactive=True,
                x=0.98,          
                xanchor="right", 
                y=0.98,          
                yanchor="top"    
            )
        ]
    )

    # LAYER3: LOCALITY OF INTEREST
    interest_geom = (
        localities
        .loc[localities['locality'] == locality_of_interest, 'geometry']
        .to_crs(localities.crs)
        .iloc[0]
    )
    interest_gdf = gpd.GeoDataFrame(
            {'locality': [locality_of_interest], 'geometry': [interest_geom]},
            crs=localities.crs
        )
    
    fig.add_trace(
        go.Choroplethmapbox(
            geojson=interest_gdf.__geo_interface__,
            locations=interest_gdf.index,
            z=[1],  # dummy, just to enable color
            showscale=False,  
            marker_opacity=0.7,
            marker_line_width=1.5,
            marker_line_color='black',
            colorscale=[[0, 'gray'], [1, 'gray']], 
            name=locality_of_interest,
            hovertemplate=f"<b>{locality_of_interest}</b><br>Locality of interest<extra></extra>"
        )
    )

    fig = apply_plotly_style(
        fig,
        title=f'Percentage Drop in Number of Trips per Day to {locality_of_interest} on Weekend',
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig.show(config=config)
    fig.write_html(f"{FIGURE_PATH}/weekday_weekend_service_drop.html", config=config)


plot_weekday_weekend_service_drop(df=data, localities=localities, locality_of_interest=locality_of_interest, ANALYSIS_PERIOD=DEFAULT_ANALYSIS_PERIOD)

# Accessibility Quadrants

In [ ]:
def plot_accessibility_quadrants(df, localities, locality_of_interest, ANALYSIS_PERIOD):
    """
    Computes and plots accessibility for all origins serving locality of interest
    """
    config = {'scrollZoom': True}
    # Filter to locality of interest
    df = df[(df["to_locality"] == locality_of_interest)&(df["Day"] == "Weekday")]

    summary = (
        df.groupby(["locality", "locality_code", "mode"])
        .agg(
            weekday_trips=("number_of_trips_per_day", "sum"),
            travel_time_median=("travel_time_median", "median")
        )
        .reset_index()
    )

    summary["total_weekday_trips"] = summary.groupby("locality")["weekday_trips"].transform("sum")
    summary["fastest_travel_time"] = summary.groupby("locality")["travel_time_median"].transform("min")
    summary_unique = summary.drop_duplicates(subset=["locality"]).copy()

    summary_unique = summary_unique.merge(
        localities[["loc_code", "geometry"]],
        left_on="locality_code",
        right_on="loc_code",
        how="left"
    )

    summary_unique = gpd.GeoDataFrame(summary_unique, crs=localities.crs)

    #Compute quadrant thresholds (medians)
    tt_thresh = summary_unique["fastest_travel_time"].median()
    freq_thresh = summary_unique["total_weekday_trips"].median()

    #Classify quadrants
    def classify(row):
        if row["fastest_travel_time"] <= tt_thresh and row["total_weekday_trips"] >= freq_thresh:
            return "Fast + Frequent"
        elif row["fastest_travel_time"] <= tt_thresh and row["total_weekday_trips"] < freq_thresh:
            return "Fast + Infrequent"
        elif row["fastest_travel_time"] > tt_thresh and row["total_weekday_trips"] >= freq_thresh:
            return "Slow + Frequent"
        else:
            return "Slow + Infrequent"

    summary_unique["quadrant"] = summary_unique.apply(classify, axis=1)
    summary_unique["quadrant_code"] = summary_unique["quadrant"].astype("category").cat.codes

    quadrant_colors = {
        "Fast + Frequent": "#1a9850",
        "Fast + Infrequent": "#91cf60",
        "Slow + Frequent": "#d5ad3d",
        "Slow + Infrequent": "#d73027"
    }
    
    quadrant_colorscale = [
        [0.0, quadrant_colors["Fast + Frequent"]],
        [0.33, quadrant_colors["Fast + Infrequent"]],
        [0.66, quadrant_colors["Slow + Frequent"]],
        [1.0, quadrant_colors["Slow + Infrequent"]],
    ]

    # Get map center
    centre_lat = localities[localities["locality"] == locality_of_interest].geometry.centroid.y.values[0]
    centre_lon = localities[localities["locality"] == locality_of_interest].geometry.centroid.x.values[0]

    # Map
    fig_map = go.Figure()

    fig_map.add_trace(
        go.Choroplethmapbox(
            geojson=summary_unique.__geo_interface__,
            locations=summary_unique.index,
            z=summary_unique["quadrant_code"],
            colorscale=quadrant_colorscale,
            marker_opacity=0.85,
            marker_line_width=0.5,
            customdata=summary_unique[["locality", "quadrant"]].values,
            hovertemplate=(
                "Locality: %{customdata[0]}<br>"
                "Accessibility: %{customdata[1]}"
                "<extra></extra>"
            ),
            showscale=False
        )
    )

    # Add fake traces for legend
    for quadrant, color in quadrant_colors.items():
        fig_map.add_trace(
            go.Scattermapbox(
                lat=[None],  # dummy
                lon=[None],  # dummy
                mode="markers",
                marker=dict(size=10, color=color),
                name=quadrant,
                showlegend=True
            )
        )

    fig_map.update_layout(
        mapbox=dict(
            center={"lat": centre_lat, "lon": centre_lon},
            zoom=8,
            style="carto-positron"
        )
    )

    # LOCALITY OF INTEREST
    interest_geom = (
        localities
        .loc[localities['locality'] == locality_of_interest, 'geometry']
        .to_crs(localities.crs)
        .iloc[0]
    )
    interest_gdf = gpd.GeoDataFrame(
            {'locality': [locality_of_interest], 'geometry': [interest_geom]},
            crs=localities.crs
        )
    
    fig_map.add_trace(
        go.Choroplethmapbox(
            geojson=interest_gdf.__geo_interface__,
            locations=interest_gdf.index,
            z=[1],  # dummy, just to enable color
            showscale=False,  
            marker_opacity=0.7,
            marker_line_width=1.5,
            marker_line_color='black',
            colorscale=[[0, 'gray'], [1, 'gray']], 
            name=locality_of_interest,
            hovertemplate=f"<b>{locality_of_interest}</b><br>Locality of interest<extra></extra>"
        )
    )


    fig_map = apply_plotly_style(
        fig_map,
        title="Accessibility Classification of Localities (Weekday)",
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig_map.show(config=config)
    fig_map.write_html(f"{FIGURE_PATH}/locality_quadrants_map.html", config=config)

    # Scatter plot
    fig_scatter = go.Figure()

    for quadrant, color in quadrant_colors.items():
        df_q = summary_unique[summary_unique["quadrant"] == quadrant]

        fig_scatter.add_trace(
            go.Scatter(
                x=df_q["fastest_travel_time"],
                y=df_q["total_weekday_trips"],
                mode="markers",
                name=quadrant,
                marker=dict(size=6, color=color),
                text=df_q["locality"],
                hovertemplate=(
                    "Locality: %{text}<br>"
                    "Accessibility: " + quadrant + "<br>"
                    "Fastest Travel Time: %{x} min<br>"
                    "Trips per Day: %{y}<extra></extra>"
                )
            )
        )

    # Vertical threshold
    fig_scatter.add_trace(
        go.Scatter(
            x=[tt_thresh, tt_thresh],
            y=[
                summary_unique["total_weekday_trips"].min(),
                summary_unique["total_weekday_trips"].max()
            ],
            mode="lines",
            line=dict(color="gray", dash="dash"),
            name="Median",
            legendgroup="median",
            showlegend=True,
        )
    )

    # Horizontal threshold
    fig_scatter.add_trace(
        go.Scatter(
            x=[
                summary_unique["fastest_travel_time"].min(),
                summary_unique["fastest_travel_time"].max()
            ],
            y=[freq_thresh, freq_thresh],
            mode="lines",
            line=dict(color="gray", dash="dash"),
            name="Median",
            legendgroup="median",
            showlegend=False,
        )
    )


    fig_scatter.update_xaxes(title="Fastest Travel Time (min)")
    fig_scatter.update_yaxes(title="Total Weekday Trips")

    fig_scatter = apply_plotly_style(
        fig_scatter,
        title="Accessibility Classification (Weekday)",
        x_title="Fastest Travel Time (min)",
        y_title=f"Trips To {locality_of_interest} per Day",
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig_scatter.show()
    fig_scatter.write_html(f"{FIGURE_PATH}/locality_quadrants_scatter.html")


    # pie chart
    quadrant_counts = (
        summary_unique.groupby("quadrant")["locality"]
        .nunique()
        .sort_index()
    )

    fig_pie = go.Figure(
        go.Pie(
            labels=quadrant_counts.index,
            values=quadrant_counts.values,
            marker=dict(colors=[quadrant_colors[q] for q in quadrant_counts.index]),
            textinfo="label+percent",
            name="Quadrant Share",
            hovertemplate=(
            "Accessibility: %{label}<br>"
            "Number of Localities: %{value}<br>"
            "Percentage of Localities: %{percent}<extra></extra>"
            )
        )
    )

    fig_pie.update_layout(showlegend=False)


    fig_pie = apply_plotly_style(
        fig_pie,
        title="Accessibility Classification (Weekday)",
        analysis_period_text=ANALYSIS_PERIOD
    )

    fig_pie.show()
    fig_pie.write_html(f"{FIGURE_PATH}/locality_quadrants_pie.html")

    locality_mode_combo = (
        df.groupby(['locality', 'locality_code'])['mode']
        .apply(
            lambda x: (
                f"{list(set(x))[0]} Only"
                if len(set(x)) == 1
                else ' + '.join(sorted(set(x)))
            )
        )
        .reset_index(name='mode_combo')
    )

    summary_unique = summary_unique.merge(
        locality_mode_combo,
        left_on="locality_code",
        right_on="locality_code",
        how="left"
    )

    

    mode_quadrant_counts = (
        summary_unique.groupby(["mode_combo", "quadrant"])["locality_code"]
        .nunique()
        .sort_index()
    ).reset_index()


    # Plot the number of distinct localities directly connected to the locality of interest by mode of transport.
    fig = px.bar(
        mode_quadrant_counts,
        x="mode_combo",
        y="locality_code",
        color="quadrant",
        barmode="group",
        color_discrete_map=quadrant_colors,
        labels={"mode_combo": "Mode", "locality_code": "Number of Localities","quadrant": "Accessibility"},
        custom_data=["quadrant"],
    )
    fig.update_traces(
        hovertemplate=(
            "Mode: %{x}<br>"
            "Accessibility: %{customdata[0]}<br>"
            "Number of Localities: %{y}<br>"
        )
    )

    fig = apply_plotly_style(
        fig,
        title="Accessibility Classification (Weekday)",
        x_title="Mode",
        y_title="Number of Localities",
        analysis_period_text=ANALYSIS_PERIOD
    )


    fig.show()
    fig.write_html(f"{FIGURE_PATH}/locality_quadrants_mode.html")
    



plot_accessibility_quadrants(data, localities, locality_of_interest, DEFAULT_ANALYSIS_PERIOD)